#  🤗 Transformers Finetuning

__Автор задач: Блохин Н.В. (NVBlokhin@fa.ru)__

Материалы:
* https://huggingface.co/docs/transformers/training
* https://huggingface.co/docs/datasets/main/en/repository_structure
* https://huggingface.co/docs/datasets/main/en/package_reference/loading_methods#datasets.load_dataset
* https://huggingface.co/docs/transformers/v4.35.2/en/training#prepare-a-dataset
* https://huggingface.co/docs/datasets/process
* https://huggingface.co/docs/evaluate/index
* https://huggingface.co/docs/transformers/main_classes/trainer
* https://huggingface.co/docs/transformers/v4.35.2/en/main_classes/trainer#transformers.TrainingArguments

## Задачи для совместного разбора

1\. Обсудите основные шаги по дообучению моделей из экосистемы 🤗 Transformers.

## Задачи для самостоятельного решения

In [ ]:
!pip install transformers[sentencepiece]
!pip install sacremoses
!pip install torchmetrics
!pip install -U sentence-transformers
!pip install datasets
!pip install evaluate
!pip install --upgrade accelerate

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from datasets import load_dataset
from transformers import pipeline, AutoTokenizer, AutoModelForSequenceClassification
import sentencepiece
from torchtext.data.metrics import bleu_score
from tqdm import tqdm
import torch
from sklearn.metrics import f1_score
import numpy as np
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim
import torchmetrics as M
from transformers import TrainingArguments, Trainer
import evaluate

device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


<p class="task" id="1"></p>

1\. Разбейте данные из файла `reviews_polarity.csv` на обучающее и валидационное множество в соотношении 80 на 20. Создайте папку `reviews_polarity_dataset` и сохраните в нее полученные фрагменты данных под названием `train.csv` и `test.csv`. Создайте объект `datasets.Dataset`, используя функцию `load_dataset`.

Токенизируйте строки при помощи токенизатора, соотвествующего модели `rubert-base-cased-sentiment`. Удалите из датасета поле `text` после токенизации, замените поле `class` на `labels` и приведите данные к тензорам `torch`.

Создайте два `DataLoader` на основе обучающего и валидационного множества. Получите батч из обучающего множества и выведите его на экран.

- [ ] Проверено на семинаре

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/Учеба/nlp/5_transformers/reviews_polarity.csv')
train, test = train_test_split(data, test_size=0.2)
train.to_csv('/content/drive/MyDrive/Учеба/nlp/5_transformers/reviews_polarity_dataset/train.csv')
test.to_csv('/content/drive/MyDrive/Учеба/nlp/5_transformers/reviews_polarity_dataset/test.csv')

In [ ]:
data_files = {'train': 'train.csv', 'test': 'test.csv'}
dset = load_dataset('/content/drive/MyDrive/Учеба/nlp/5_transformers/reviews_polarity_dataset', data_files=data_files).rename_column('class','labels')
dset

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'text', 'labels'],
        num_rows: 30574
    })
    test: Dataset({
        features: ['Unnamed: 0', 'text', 'labels'],
        num_rows: 7644
    })
})

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("blanchefort/rubert-base-cased-sentiment", device=device)

dset = dset.map(
  lambda x: tokenizer(x, return_tensors='pt', padding=True),
  input_columns=['text'],
  batched=True,
  batch_size=40000
).remove_columns(['Unnamed: 0', 'text']).with_format("torch", columns=['labels', 'input_ids', 'token_type_ids', 'attention_mask'])#.map(torch.stack, input_columns=['input_ids'])
dset

tokenizer_config.json:   0%|          | 0.00/499 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/943 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/1.40M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Map:   0%|          | 0/30574 [00:00<?, ? examples/s]

Map:   0%|          | 0/7644 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 30574
    })
    test: Dataset({
        features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 7644
    })
})

In [ ]:
train_loader = DataLoader(dset['train'], batch_size=32, drop_last=True)
test_loader = DataLoader(dset['test'], batch_size=32, drop_last=False)

for i in train_loader:
  print(i)
  break

{'labels': tensor([0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1]), 'input_ids': tensor([[  101,  1067,  3852,  ...,     0,     0,     0],
        [  101, 52148,  3418,  ...,     0,     0,     0],
        [  101, 23260,   336,  ...,     0,     0,     0],
        ...,
        [  101,  8741, 18358,  ...,     0,     0,     0],
        [  101,  1468,   801,  ...,     0,     0,     0],
        [  101, 49850,   322,  ...,     0,     0,     0]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])}


<p class="task" id="2"></p>

2\. Создайте модель при помощи класса `AutoModelForSequenceClassification`, заменив голову модели в соответствии с задачей бинарной классификации. Используя стандартный цикл обучения `torch`, настройте модель для решения задачи бинарной классификации. Во время обучения выводите на экран значение функции потерь (используйте готовые значения, которые генерирует модель) на обучающем множестве и f1 на валидационном множестве.

Здесь и далее для ускорения процесса обучения вы можете заморозить часть сети или уменьшить размер наборов данных, выбрав небольшое подмножество примеров.

- [ ] Проверено на семинаре

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained("blanchefort/rubert-base-cased-sentiment").to(device)
for param in model.base_model.parameters():
  param.requires_grad = False

in_features = model.classifier.in_features
model.classifier = nn.Linear(
  in_features = in_features,
  out_features=2
)
model = model.to(device)

model.safetensors:   0%|          | 0.00/711M [00:00<?, ?B/s]

In [ ]:
criterion = nn.CrossEntropyLoss().cuda()
optimizer = optim.Adam(model.parameters(), lr=0.001)
n_epochs = 5

epoch_losses = []
for epoch in range(n_epochs):
  model.train()
  batch_loss = []
  f1_test = M.F1Score(task='binary').to(device)
  for batch in tqdm(train_loader):
    X, y = {'input_ids': batch['input_ids'].to(device),  'token_type_ids': batch['token_type_ids'].to(device), 'attention_mask': batch['attention_mask'].to(device)}, batch['labels'].to(device)
    out = model(**X)
    preds = out.logits
    loss = criterion(preds, y)

    batch_loss.append(loss.item())

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

  epoch_losses.append(np.mean(batch_loss))
  print(f'Epoch: {epoch+1}, loss: {epoch_losses[-1]}')

  model.eval()
  for batch in test_loader:
    X, y = {'input_ids': batch['input_ids'].to(device),  'token_type_ids': batch['token_type_ids'].to(device), 'attention_mask': batch['attention_mask'].to(device)}, batch['labels'].to(device)
    outputs = model(**X)
    preds = outputs.logits
    f1_test.update(preds.argmax(dim=1), y)

  print(f'f1score test={f1_test.compute()}')
  print()

100%|██████████| 955/955 [01:33<00:00, 10.23it/s]


Epoch: 1, loss: 0.3870479239609229
f1score test=0.897718071937561



100%|██████████| 955/955 [01:36<00:00,  9.85it/s]


Epoch: 2, loss: 0.38196797516801595
f1score test=0.8970019221305847



100%|██████████| 955/955 [01:36<00:00,  9.88it/s]


Epoch: 3, loss: 0.3817752047359007
f1score test=0.8971962332725525



100%|██████████| 955/955 [01:36<00:00,  9.88it/s]


Epoch: 4, loss: 0.3812058102398019
f1score test=0.8984779715538025



100%|██████████| 955/955 [01:36<00:00,  9.88it/s]


Epoch: 5, loss: 0.3814385678599642
f1score test=0.8973217606544495



<p class="task" id="3"></p>

3\. Создайте модель при помощи класса `AutoModelForSequenceClassification`, заменив голову модели в соответствии с задачей бинарной классификации. Используя `transformers.Trainer`, настройте модель для решения задачи бинарной классификации. При настройке `Trainer` укажите количество эпох, равное 5. Во время обучения выводите на экран значение функции потерь на обучающем множестве и f1 на валидационном множестве.  

- [ ] Проверено на семинаре


In [ ]:
class Net(nn.Module):
  def __init__(self):
    super(Net, self).__init__()
    self.model = AutoModelForSequenceClassification.from_pretrained("blanchefort/rubert-base-cased-sentiment")
    for param in self.model.base_model.parameters():
      param.requires_grad = False
    in_features = self.model.classifier.in_features
    self.model.classifier = nn.Linear(in_features=in_features,
                                      out_features=2)
    self.loss = nn.CrossEntropyLoss()

  def forward(self, input_ids, labels=None):
    outputs = self.model(input_ids).logits
    loss = self.loss(outputs, labels)
    return loss, outputs

In [ ]:
metric = evaluate.load("f1")
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=1)
    return metric.compute(predictions=preds, references=labels)

args = TrainingArguments(output_dir="test_trainer",
                         evaluation_strategy="epoch",
                         num_train_epochs=5)

In [ ]:
model = Net()
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=dset['train'],
    eval_dataset=dset['test'],
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,F1
1,0.477000,0.469698,0.884413
2,0.472200,0.472379,0.884413
3,0.456200,0.469777,0.884413
4,0.464000,0.465548,0.884413
5,0.484200,0.467760,0.884413


TrainOutput(global_step=19110, training_loss=0.46816520092064395, metrics={'train_runtime': 978.8584, 'train_samples_per_second': 156.172, 'train_steps_per_second': 19.523, 'total_flos': 0.0, 'train_loss': 0.46816520092064395, 'epoch': 5.0})

<p class="task" id="4"></p>

4\. Используя эмбеддинги `distiluse-base-multilingual-cased-v1` из пакета `sentence_transformers`, решите задачу бинарной классификации. Для этого добавьте несколько полносвязных слоев поверх модели `SentenceTransformer`. Заморозьте часть модели, отвечающей за генерацию эмбеддингов. Во время обучения выводите на экран значение функции потерь на обучающем множестве и f1 на валидационном множестве.  

- [ ] Проверено на семинаре

In [ ]:
from sentence_transformers import SentenceTransformer
class Net(nn.Module):
  def __init__(self):
    self.base_model = SentenceTransformer('..')
    self.classifier = nn.Sequential(
        nn.Linear(?, 64),
        nn.ReLU(),
        nn.Linear(64, 2)
    )

## Обратная связь
- [ ] Хочу получить обратную связь по решению